In [1]:
# __getattribute__(self, name)  -> it's a method bound to the instance
# check https://blog.ionelmc.ro/2015/02/09/understanding-python-metaclasses/ for a flow graph, it's a bit simplitied but it shows what's what


class Person:
    pass

In [2]:
p = Person()

try:
    p.name()
except AttributeError as e:
    print(type(e), e)


<class 'AttributeError'> 'Person' object has no attribute 'name'


In [3]:
class Person:
    def __getattr__(self, name):
        print("Called __getattr__ with:", name)
        return "Not Found"

p = Person()
p.first_name

Called __getattr__ with: first_name


'Not Found'

In [4]:
class Person:
    def __getattr__(self, name):
        print("Called __getattr__ with:", name)
        alt_name = "_" + name
        if getattr(self, alt_name, None) is not None:  # causes recursion error
            return getattr(self, alt_name)
        raise AttributeError(f"Could not find `{name}` or `{alt_name}`")


p = Person()
try:
    p.age  # causes infinite recursion error caused by calling getattr inside `__getattr__` 
except Exception as e:
    print(type(e), e)

Called __getattr__ with:

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [5]:
class Person:
    def __getattr__(self, name):
        alt_name = "_" + name
        print("Called __getattr__ with:", name, alt_name)
        try:
            return super().__getattribute__(alt_name)  # ask super() if attr exists, will not cause recursion error
        except AttributeError as e:
            raise AttributeError(f"Could not find {name} or {alt_name}")


p = Person()
try:
    p.age
except AttributeError as e:
    print(type(e), e)


Called __getattr__ with: age _age
<class 'AttributeError'> Could not find age or _age


In [6]:
p.__dict__["_age"] = 33
p.age


Called __getattr__ with: age _age


33

In [7]:
class DefaultClass:
    """Set attribute name with default value if requested attr does not exists."""
    def __init__(self, attribute_default=None):
        self._attribute_default = attribute_default

    def __getattr__(self, name):
        print(f"`{name}` not found, creating it and setting it to default: `{self._attribute_default}`")
        setattr(self, name, self._attribute_default)
        return self._attribute_default
        

In [8]:
d = DefaultClass("NotAvailable")
d.test

`test` not found, creating it and setting it to default: `NotAvailable`


'NotAvailable'

In [9]:
d.__dict__

{'_attribute_default': 'NotAvailable', 'test': 'NotAvailable'}

In [10]:
d.age

`age` not found, creating it and setting it to default: `NotAvailable`


'NotAvailable'

In [11]:
d.__dict__

{'_attribute_default': 'NotAvailable',
 'test': 'NotAvailable',
 'age': 'NotAvailable'}

In [12]:
class Person(DefaultClass):
    def __init__(self, name):
        super().__init__(attribute_default="Attr not found")
        self.name = name
        

In [13]:
p = Person("Bob")
p.age

`age` not found, creating it and setting it to default: `Attr not found`


'Attr not found'

In [14]:
p.__dict__

{'_attribute_default': 'Attr not found',
 'name': 'Bob',
 'age': 'Attr not found'}

In [15]:
class AttributeNotFoundLogger:
    def __getattr__(self, name):
        error_message = f"{type(self).__name__} object has no attribure {name}"
        print(f"Log: {error_message}")
        raise AttributeError(error_message)


class Person(AttributeNotFoundLogger):
    def __init__(self, name):
        self.name = name


In [16]:
p = Person("Alex")
p.name

'Alex'

In [17]:
try:
    p.age
except AttributeError as e:
    # error gets "logged"
    print(type(e), e)

Log: Person object has no attribure age
<class 'AttributeError'> Person object has no attribure age


In [18]:
class Person:
    def __init__(self, name, age):
        self._name = name
        self._age = age

    def __getattr__(self, name):
        # we would expect it to throw the error, but please remember the attribute lookup order!
        # __dict__ will be checked before __getattr__
        if name.startswith("_"):
            raise AttributeError(f"Access to private attribute {name} is forbidden! Use public names instead.")
        return super().__getattribute__(name)

In [19]:
p = Person("Bob", 33)
try:
    p.name
except AttributeError as e:  # expected
    print(type(e), e)


<class 'AttributeError'> 'Person' object has no attribute 'name'


In [20]:
# but now
print(p.__dict__)
try:
    p._name
except AttributeError as e:  # will this throw an error? -> NO, instance.__dict__ is checked before calling __getattr__
    print(type(e), e)

{'_name': 'Bob', '_age': 33}


In [21]:
class Person:
    def __init__(self, name, age):
        self._name = name
        self._age = age

    def __getattribute__(self, name):
        # this will in fact throw the error as expected
        if name.startswith("_"):
            raise AttributeError(f"Access to private attribute {name} is forbidden! Use public names instead.")
        return super().__getattribute__(name)

In [22]:
p = Person("Bob", 35)
try:
    p._name
except AttributeError as e:
    print(type(e), e)

<class 'AttributeError'> Access to private attribute _name is forbidden! Use public names instead.


In [23]:
# but also
try:
    p.__dict__
except AttributeError as e:
    print(type(e), e)

<class 'AttributeError'> Access to private attribute __dict__ is forbidden! Use public names instead.


In [24]:
class Person:
    def __init__(self, name, age):
        self._name = name
        self._age = age

    def __getattribute__(self, name):
        # this will in fact throw the error as expected
        if name.startswith("_") and not name.startswith("__"):
            raise AttributeError(f"Access to private attribute {name} is forbidden! Use public names instead.")
        return super().__getattribute__(name)

In [25]:
p = Person("John", 52)
p.__dict__  # now works fine

{'_name': 'John', '_age': 52}

In [26]:
class Person:
    def __init__(self, name, age):
        self._name = name
        self._age = age

    def __getattribute__(self, name):
        # this will in fact throw the error as expected
        if name.startswith("_") and not name.startswith("__"):
            raise AttributeError(f"Access to private attribute {name} is forbidden! Use public names instead.")
        return super().__getattribute__(name)

    @property
    def name(self):
        # so now we want to accecss the "protected" attr that is guarded in our __getattribure__ ;) Will it work?
        return self._name

    @property
    def age(self):
        return self._age


In [27]:
p = Person("Bob", 33)
try:
    p.name
except AttributeError as e:
    print(type(e), e)

<class 'AttributeError'> Access to private attribute _name is forbidden! Use public names instead.


In [28]:
class Person:
    def __init__(self, name, age):
        self._name = name
        self._age = age

    def __getattribute__(self, name):
        # this will in fact throw the error as expected
        if name.startswith("_") and not name.startswith("__"):
            raise AttributeError(f"Access to private attribute {name} is forbidden! Use public names instead.")
        return super().__getattribute__(name)

    @property
    def name(self):
        print("super().__getattribute__('_name')")
        return super().__getattribute__("_name")

    @property
    def age(self):
        print("super().__getattribute__('_age')")
        return super().__getattribute__("_age")


In [29]:
p = Person("Mike", 43)
p.name, p.age

super().__getattribute__('_name')
super().__getattribute__('_age')


('Mike', 43)

In [30]:
p._test = "test"  # setting works without problems
p.__dict__

{'_name': 'Mike', '_age': 43, '_test': 'test'}

In [31]:
class DefaultClass:
    """Set attribute name with default value if requested attr does not exists."""
    def __init__(self, attribute_default=None):
        self._attribute_default = attribute_default

    def __getattr__(self, name):
        default_value = super().__getattribute__("_attribute_default")
        print(f"`{name}` not found, creating it and setting it to default: `{default_value}`")
        setattr(self, name, default_value)
        return default_value

In [32]:
class Person(DefaultClass):
    def __init__(self, name, age):
        super().__init__(attribute_default="Not found!")
        if name:
            self._name = name
        if age:
            self._age = age

    def __getattribute__(self, name):
        # this will in fact throw the error as expected
        if name.startswith("_") and not name.startswith("__"):
            raise AttributeError(f"Access to private attribute {name} is forbidden! Use public names instead.")
        return super().__getattribute__(name)

    @property
    def name(self):
        print("super().__getattribute__('_name')")
        return super().__getattribute__("_name")

    @property
    def age(self):
        print("super().__getattribute__('_age')")
        return super().__getattribute__("_age")

In [33]:
p = Person("Bob", 33)
p.age, p.name

super().__getattribute__('_age')
super().__getattribute__('_name')


(33, 'Bob')

In [34]:
p.first_name

`first_name` not found, creating it and setting it to default: `Not found!`


'Not found!'

In [35]:
p.__dict__

{'_attribute_default': 'Not found!',
 '_name': 'Bob',
 '_age': 33,
 'first_name': 'Not found!'}

In [36]:
class MetaLogger(type):
    def __getattribute__(self, name):
        print(f"Log {type(self).__name__} __getattribure__ name=`{name}`")
        return super().__getattribute__(name)

    def __getattr__(self, name):
        print(f"Log {type(self).__name__} __getattr__ name=`{name}`")
        return "Default value"
        

In [37]:
class Account(metaclass=MetaLogger):
    apr = 10


In [38]:
Account.apr

Log MetaLogger __getattribure__ name=`apr`


10

In [39]:
Account.number

Log MetaLogger __getattribure__ name=`number`
Log MetaLogger __getattr__ name=`number`


'Default value'

In [40]:
a = Account()
a.apr

10

In [41]:
try:
    a.xyz
except AttributeError as e:
    print(type(e), e)

<class 'AttributeError'> 'Account' object has no attribute 'xyz'


In [42]:
class MyClass:
    def __getattribute__(self, name):
        print(f"MyClass.__getattribure__ name={name}")
        return super().__getattribute__(name)

    def __getattr__(self, name):
        print(f"MyClass.__getattr__ name={name}")
        raise AttributeError("MyClass.__getattr__")

    def say_hello(self):
        return "Hello"

In [43]:
m = MyClass()
m.say_hello()

MyClass.__getattribure__ name=say_hello


'Hello'